In [1]:
import os
import numpy as np
from itertools import combinations
import random
import logging
from typing import Dict, Tuple, List
from Metabackbone_functions import (
    load_dna_structure_files, find_longest_strand, find_cross_over_in_longest_strand, 
    calculate_left_right_pos, find_valid_point, find_bases_around_point, 
    calculate_center_of_mass, calculate_bend_angle, find_bend_angle, 
    find_bases_in_sphere, remove_three_strands_in_sphere, export_dna_structures, 
    run_all_simulations, stored_removed_strands)
from ipy_oxdna.dna_structure import DNAStructure, DNAStructureStrand, load_dna_structure, DNABase, strand_from_info
from ipy_oxdna.oxdna_simulation import Simulation, SimulationManager
from ER_functions import run_simulations_for_structure, load_simulated_structure, evaluate_fitness, check_dna_structure, create_index_position_map, get_indexes_from_positions, update_indices_for_selected_structures

In [2]:
# ANSI color codes for printing colored messages
colors = {
    'blue': '34',
    'green': '32',
    'yellow': '33',
    'cyan': '36',
    'red': '31',
    'magenta': '35'
}


In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO)

def print_colored(message, color_code):
    print(f"\033[{color_code}m{message}\033[0m")
    

In [4]:
left_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181,2182,2183,2184,2185,2186,2187,2188,2189,2190,2191,2192,2193,2194,2195,2196,2197,2198,2199,2200,2201,2202,2203,2204,2205,3269,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279,3280,3281,3282,3283,3284,3285,3296,3302,3303,3304,3305,3306,3307,3308,3309,3310,3311,3312,3313,3314,3315,3316,3317,3318,3319,3320,3321,3322,3323,3324,3325,3326,3327,3328,3329,3330,3331,3332,3333,3334,3335,3336,3337,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366,3367,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377,3378,3379,3380,3381,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415,3416,3417,3418,3419,3420,3421,3422,3423,3424,3425,3426,3427,3428,3429,3430,3431,3432,3433,3434,3435,3436,3437,3438,3439,3440,3441,3442,3443,3444,3445,3446,3447,3448,3449,3450,3451,3452,3453,3454,3455,3456,3457,3458,3459,3460,3461,3462,3463,3464,3465,3466,3467,3468,3469,3470,3471,3472,3473,3474,3475,3476,3477,3478,3479,3480,3481,3482,3483,3484,3485,3486,3487,3488,3489,3490,3491,3492,3493,3494,3495,3496,3497,3498,3499,3500,3501,3502,3503,3504,3505,3506,3507,3508,3509,3510,3511,3512,3513,3514,3515,3516,3517,3518,3519,3520,3521,3522,3523,3524,3525,3526,3527,3528,3529,3530,3531,3532,3533,3534,3535,3536,3537,3538,3539,3540,3541,3542,3543,3544,3545,3546,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556,3557,3558,3559,4066,4101,4102,4103,4104,4105,4106,4107,4108,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118,4119,4120,4121,4122,4123,4124,4125,4126,4127,4128,4129,4130,4131,4132,4133,4134,4135,4136,4137,4138,4139,4140,4141,4142,4143,4144,4145,4146,4147,4148,4149,4150,4151,4152,4153,4154,4155,4156,4157,4158,4159,4160,4161,4162,4163,4164,4165,4166,4167,4168,4169,4170,4171,4172,4173,4174,4175,4176,4177,4178,4179,4180,4181,4182,4183,4184,4185,4186,4187,4188,4189,4190,4191,4192,4193,4194,4195,4196,4197,4198,4199,4200,4201,4202,4203,4204,4205,4469,4470,4471,4472,4473,4474,4475,4479,4480,4481,4482,4483,4484,4485,4486,4487,4488,4489,4490,4491,4492,4493,4494,4495,4496,4497,4498,4499,4500,4501,4502,4503,4504,4505,4506,4507,4508,4509,4510,4511,4512,4513,4514,4515,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525,4526,4527,4528,4529,4530,4531,4532,4533,4534,4535,4536,4537,4538,4539,4540,4541,4542,4543,4544,4545,4546,4547,4548,4549,4550,4551,4552,4553,4554,4555,4556,4557,4558,4559,4560,4561,4562,4563,4564,4565,4566,4567,4568,4569,4570,4571,4572,4573,4574,4575,4576,4577,4578,4579,4580,4581]
right_indices = [20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,2312,2313,2314,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400,2401,2402,2403,2404,2405,2406,2407,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419,2420,2421,2422,2423,2424,2425,2426,2427,2428,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450,2451,2452,2453,2454,2455,2456,2457,2458,2459,2460,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500,2501,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2527,2528,2529,2530,2531,2532,2533,3560,3561,3562,3563,3564,3565,3566,3567,3568,3569,3570,3571,3572,3573,3574,3575,3576,3577,3578,3579,3580,3581,3582,3583,3584,3585,3586,3587,3588,3589,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599,3600,3601,3602,3603,3604,3605,3606,3607,3608,3609,3610,3611,3612,3613,3614,3615,3616,3617,3618,3619,3620,3621,3622,3623,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633,3634,3635,3636,3637,3638,3639,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653,3654,3655,3656,3657,3658,3659,3660,3661,3662,3663,3664,3665,3666,3667,3668,3669,3670,3671,3672,3673,3674,3675,3676,3677,3678,3679,3680,3682,3683,3684,3685,3686,3687,3688,3689,3690,3691,3692,3693,3694,3695,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705,3706,3707,3708,3709,3710,3711,3712,3713,3714,3715,3750,4206,4207,4208,4209,4210,4211,4212,4213,4214,4215,4216,4217,4218,4219,4220,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230,4231,4232,4233,4234,4235,4236,4237,4238,4239,4240,4241,4242,4243,4244,4245,4246,4247]
sphere_radius = 3.0
eq_steps = 1e3
prod_steps = 1e3
rel_steps = 1e2
min_distance_threshold = 2.5
min_distance = 7.0
max_distance = 20.0
num_best_structures = 5

eq_parameters = {'dt':f'0.003','steps':f'{eq_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                 'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}

prod_parameters = {'dt':f'0.003','steps':f'{prod_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                   'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}
rel_parameters = {'steps': f'{rel_steps}', 'max_backbone_force': '200', 'max_backbone_force_far': '200'}

In [5]:
num_iterations = 2
sphere_radius = 3.0
desired_angle = 100.0
tolerance = 5.0

input_path = "/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/unmodified/2268_bp"
base_path = "/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp"
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/Results/optimization_2268_bp'


In [6]:
def load_dna_structure_files(input_path):
    dat_path = os.path.join(input_path, '2268_bp.dat')
    top_path = os.path.join(input_path, '2268_bp.top')
    dna = load_dna_structure(top_path, dat_path)
    return dna

In [7]:
initial_dna_structure = load_dna_structure_files(input_path)

In [8]:
longest_strand, _ = find_longest_strand(initial_dna_structure)

In [9]:
point_pos = find_valid_point(initial_dna_structure, left_indices, right_indices, longest_strand)
print('point_pos:', point_pos)

Left indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 

In [10]:
mutants, removed_strands_info = remove_three_strands_in_sphere(initial_dna_structure, point_pos, sphere_radius)
print('mutants:',mutants)

Bases in sphere: [874, 875, 876, 877, 878, 1085, 1086, 1087, 1088, 1089, 1241, 1242, 1243, 1244, 1245, 1246, 1471, 1472, 1473, 1474, 1475, 1476, 1638, 1639, 1640, 1641, 1642, 1838, 1839, 1840, 1841, 1842, 2630, 2631, 2639, 2655, 2656, 2657, 2658, 2659, 2660, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672, 2687, 2688, 2689, 3751, 3752, 3753, 3754, 3755, 3771, 3772, 3773, 3818, 3819, 3820, 4278, 4279, 4280, 4281, 4282, 4283, 4284, 4287, 4288, 4289]
Base to strand mapping: {874: 2, 875: 2, 876: 2, 877: 2, 878: 2, 1085: 2, 1086: 2, 1087: 2, 1088: 2, 1089: 2, 1241: 2, 1242: 2, 1243: 2, 1244: 2, 1245: 2, 1246: 2, 1471: 2, 1472: 2, 1473: 2, 1474: 2, 1475: 2, 1476: 2, 1638: 2, 1639: 2, 1640: 2, 1641: 2, 1642: 2, 1838: 2, 1839: 2, 1840: 2, 1841: 2, 1842: 2, 2630: 11, 2631: 11, 2639: 12, 2655: 12, 2656: 12, 2657: 12, 2658: 12, 2659: 12, 2660: 12, 2665: 12, 2666: 12, 2667: 13, 2668: 13, 2669: 13, 2670: 13, 2671: 13, 2672: 13, 2687: 13, 2688: 13, 2689: 13, 3751: 43, 3752: 43, 3753: 43, 3754: 43, 3

In [11]:
summaries = stored_removed_strands(initial_dna_structure, removed_strands_info)
for summary, bases in summaries:
    print(summary)
    print("Removed bases:", bases)

For structure_0, staples with the indexes 11, 43, 12 were removed.
Removed bases: [2597, 2598, 2599, 2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 3751, 3752, 3753, 3754, 3755, 3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765, 3766, 3767, 3768, 3769, 3770, 3771, 3772, 3773, 3774, 3775, 3776, 3777, 3778, 3779, 3780, 3781, 3782, 3783, 3784, 3785, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666]
For structure_1, staples with the indexes 11, 43, 13 were removed.
Removed bases: [2597, 2598, 2599, 2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 

In [12]:
new_structures = []
new_structures.extend(mutants)
print('new structures:', new_structures)

new structures: [<ipy_oxdna.dna_structure.DNAStructure object at 0x79241f652ad0>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5fed10>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5fed90>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5fee90>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5fef50>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff110>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff1d0>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff2d0>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff490>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff090>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff550>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff710>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff790>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff890>, <ipy_oxdna.dna_structure.DNAStructure object at 0x79241f5ff950>, <ipy_oxd

In [13]:
num_mutants = len(new_structures)

In [14]:
export_paths = export_dna_structures(new_structures, base_path)
print_colored("Exported DNA structures.", colors['green'])
print_colored(f"Export paths: {export_paths}", colors['blue'])

Exported DNA structures.
Export paths: [{'structure_id': 0, 'dat_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp/structure_0/1512_bp_rmv_staples.dat', 'top_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp/structure_0/1512_bp_rmv_staples.top'}, {'structure_id': 1, 'dat_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp/structure_1/1512_bp_rmv_staples.dat', 'top_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp/structure_1/1512_bp_rmv_staples.top'}, {'structure_id': 2, 'dat_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/modified/modified_2268_bp/structure_2/1512_bp_rmv_staples.dat', 'top_path': '/home/ava/MetaBackbone_project/Metabackbone-scripts/str

In [15]:
removed_strands_info_all = []
mutants, removed_strands_info = remove_three_strands_in_sphere(initial_dna_structure, point_pos, sphere_radius)
removed_strands_info_all.extend(removed_strands_info)
print('removed_strands_info_all:', removed_strands_info_all)

Bases in sphere: [874, 875, 876, 877, 878, 1085, 1086, 1087, 1088, 1089, 1241, 1242, 1243, 1244, 1245, 1246, 1471, 1472, 1473, 1474, 1475, 1476, 1638, 1639, 1640, 1641, 1642, 1838, 1839, 1840, 1841, 1842, 2630, 2631, 2639, 2655, 2656, 2657, 2658, 2659, 2660, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672, 2687, 2688, 2689, 3751, 3752, 3753, 3754, 3755, 3771, 3772, 3773, 3818, 3819, 3820, 4278, 4279, 4280, 4281, 4282, 4283, 4284, 4287, 4288, 4289]
Base to strand mapping: {874: 2, 875: 2, 876: 2, 877: 2, 878: 2, 1085: 2, 1086: 2, 1087: 2, 1088: 2, 1089: 2, 1241: 2, 1242: 2, 1243: 2, 1244: 2, 1245: 2, 1246: 2, 1471: 2, 1472: 2, 1473: 2, 1474: 2, 1475: 2, 1476: 2, 1638: 2, 1639: 2, 1640: 2, 1641: 2, 1642: 2, 1838: 2, 1839: 2, 1840: 2, 1841: 2, 1842: 2, 2630: 11, 2631: 11, 2639: 12, 2655: 12, 2656: 12, 2657: 12, 2658: 12, 2659: 12, 2660: 12, 2665: 12, 2666: 12, 2667: 13, 2668: 13, 2669: 13, 2670: 13, 2671: 13, 2672: 13, 2687: 13, 2688: 13, 2689: 13, 3751: 43, 3752: 43, 3753: 43, 3754: 43, 3

In [16]:
def evolutionary_algorithm(initial_dna_structure, left_indices, right_indices, num_iterations, num_best_structures, desired_angle, tolerance, base_path, sim_base_path, sphere_radius):
    current_structures = [initial_dna_structure]
    removed_staples_dict = {}  # Dictionary to store removed staples info
    
    for iteration in range(num_iterations):
        print_colored(f"Iteration {iteration + 1}", colors['yellow'])
        
        new_structures = []
        removed_strands_info_all = []
        
        for dna in current_structures:
            # Step 1: Find a valid point in the DNA structure
            print_colored("Step 1: Finding a valid point in the DNA structure...", colors['red'])
            longest_strand, _ = find_longest_strand(dna)
            point_pos = find_valid_point(dna, left_indices, right_indices, longest_strand)
            print_colored(f'Found a valid point in the DNA structure: {point_pos}', colors['green'])
            
            # Step 2: Remove three random staples within a sphere around the point
            print_colored("Step 2: Removing three random staples within a sphere around the point...", colors['blue'])
            mutants, removed_strands_info = remove_three_strands_in_sphere(dna, point_pos, sphere_radius)
            removed_strands_info_all.extend(removed_strands_info)
            new_structures.extend(mutants)
            print_colored(f"Removed three random staples. Number of new structures: {len(mutants)}", colors['cyan'])
        
        # Determine the number of mutants based on the number of new structures created
        num_mutants = len(new_structures)
        print_colored(f"Iteration {iteration + 1}: Generated {num_mutants} new structures.", colors['blue'])
        
        # Step 3: Export new DNA structures
        print_colored("Step 3: Exporting DNA structures...", colors['magenta'])
        export_paths = export_dna_structures(new_structures, base_path)
        print_colored("Exported DNA structures.", colors['yellow'])
        print_colored(f"Export paths: {export_paths}", colors['red'])
        
        # Step 4: Simulate each modified structure
        print_colored("Step 4: Simulating each modified structure...", colors['green'])
        for export_path in export_paths:
            structure_id = export_path['structure_id']
            print_colored(f"Starting simulations for structure {structure_id}...", colors['blue'])
            run_simulations_for_structure(structure_id, base_path, sim_base_path, rel_parameters, eq_parameters, prod_parameters)
            # print_colored(f"Simulations for structure {structure_id} completed.", colors['blue'])
        
        # Step 5: Measure the angle at the joint for each mutant after simulation
        print_colored("Step 5: Measuring the angle at the joint for each mutant after simulation...", colors['cyan'])
        angles = []
        for export_path in export_paths:
            structure_id = export_path['structure_id']
            simulated_dna = load_simulated_structure(structure_id, sim_base_path)
            
            bend_angle = find_bend_angle(simulated_dna, left_indices, right_indices, longest_strand, point_pos)
            angles.append((structure_id, bend_angle))
            print_colored(f"Measured bend angle for structure {structure_id}: {bend_angle} degrees.", colors['magenta'])
        
        # Step 6: Evaluate fitness
        print_colored("Step 6: Evaluating fitness of mutants...", colors['yellow'])
        fitness_scores = evaluate_fitness([angle for _, angle in angles], desired_angle, tolerance)
        print_colored("Evaluated fitness of mutants.", colors['red'])
        print_colored(f"Fitness scores: {fitness_scores}", colors['green'])
        
        # Step 7: Select the best mutants based on fitness scores
        print_colored("Step 7: Selecting the best mutants based on fitness scores...", colors['blue'])
        sorted_mutants = sorted(zip(angles, fitness_scores), key=lambda x: x[1])
        
        # Display results for all structures
        print_colored("Results for all structures:", colors['cyan'])
        for (structure_id, angle), fitness_score in sorted_mutants:
            print_colored(f"Structure ID: {structure_id}, Bend Angle: {angle}, Fitness Score: {fitness_score}", colors['magenta'])

        # Select the best mutants
        best_mutants = [new_structures[i] for i, (_, _) in enumerate(sorted_mutants[:num_best_structures])]
        best_angles = [angle for (_, angle), _ in sorted_mutants[:num_best_structures]]
        best_removed_strands_info = [removed_strands_info_all[i] for i, (_, _) in enumerate(sorted_mutants[:num_best_structures])]
        
        # Display results for the selected best structures
        print_colored(f"Selected the best {num_best_structures} mutants:", colors['yellow'])
        for i, (angle, fitness_score) in enumerate(zip(best_angles, [fs for _, fs in sorted_mutants[:num_best_structures]])):
            print_colored(f"Selected Structure {i}: Bend Angle: {angle}, Fitness Score: {fitness_score}", colors['red'])

        # Step 8: Store the removed staples info for the best mutants
        print_colored("Step 8: Storing the removed staples info for the best mutants...", colors['green'])
        for i, (structure_id, _) in enumerate(sorted_mutants[:num_best_structures]):
            removed_staples_dict[structure_id] = best_removed_strands_info[i]
            summary, removed_bases = stored_removed_strands(initial_dna_structure, [best_removed_strands_info[i]])[0]
            print_colored(f"Structure ID: {structure_id} - {summary}", colors['blue'])
            print_colored(f"Removed bases: {removed_bases}", colors['cyan'])
        
        # Check if the best angle is within the desired tolerance
        if any(abs(angle - desired_angle) <= tolerance for angle in best_angles):
            print_colored("Desired angle achieved within tolerance. Stopping evolution process.", colors['magenta'])
            break
        
        # Step 9: Update the current structures with the best mutants for the next iteration
        print_colored("Step 9: Updating the current structures with the best mutants for the next iteration...", colors['yellow'])
        current_structures = best_mutants
        print_colored(f"Updated current structures for iteration {iteration + 1}.", colors['red'])
        
        # Step 10: Update left and right indices for each selected structure
        print_colored("Step 10: Updating left and right indices for each selected structure...", colors['green'])
        new_indices_list = update_indices_for_selected_structures(current_structures, initial_dna_structure, left_indices, right_indices, best_removed_strands_info)
        left_indices, right_indices = new_indices_list[0]
    
    print_colored("Evolutionary algorithm completed.", colors['red'])
                     

In [17]:
evolutionary_algorithm(initial_dna_structure, left_indices, right_indices, num_iterations, num_best_structures, desired_angle, tolerance, base_path, sim_base_path, sphere_radius)

Iteration 1
Step 1: Finding a valid point in the DNA structure...
Left indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544,

INFO: Lists updated 1 times (every ~100.000000 steps)
INFO: Aggregated I/O statistics (set debug=1 for file-wise information)
	  2.253 MB written to files
	 96.000  B written to stdout/stderr
	For a total of    0.538 MB/s



All queued simulations finished in: 10.52137154713273
Relaxation simulation for structure 0 completed.
Running: eq
Run time: 0.5812697419896722
Finished: eq


INFO: Lists updated 3 times (every ~333.333333 steps)
INFO: Aggregated I/O statistics (set debug=1 for file-wise information)
	  1.181 MB written to files
	 62.000  B written to stdout/stderr
	For a total of     3.74 MB/s



All queued simulations finished in: 10.507221107836813
Equilibration simulation for structure 0 completed.
Running: prod
Run time: 0.6113649690523744
Finished: prod


...

Measured bend angle for structure 15: 144.23283483338943 degrees.
Measured bend angle for structure 16: 148.56877510034053 degrees.
Measured bend angle for structure 17: 149.2819577123364 degrees.
Measured bend angle for structure 18: 146.45539610717708 degrees.
Measured bend angle for structure 19: 156.00672996985492 degrees.


Measured bend angle for structure 20: 147.65781544465375 degrees.
Measured bend angle for structure 21: 149.8091566617089 degrees.
Measured bend angle for structure 22: 147.0386302634045 degrees.
Measured bend angle for structure 23: 149.85111805991633 degrees.
Measured bend angle for structure 24: 149.37524436992504 degrees.


Measured bend angle for structure 25: 151.90160274251087 degrees.
Measured bend angle for structure 26: 149.0706036792266 degrees.
Measured bend angle for structure 27: 151.77817243570368 degrees.
Measured bend angle for structure 28: 155.4755542009308 degrees.
Measured bend angle for structure 29: 149.05847277934515 degrees.


Measured bend angle for structure 30: 145.0196208605789 degrees.
Measured bend angle for structure 31: 151.75006878637964 degrees.
Measured bend angle for structure 32: 156.36256497147917 degrees.
Measured bend angle for structure 33: 156.13251215607855 degrees.
Measured bend angle for structure 34: 149.77340474676691 degrees.


Measured bend angle for structure 35: 159.07913644866406 degrees.
Measured bend angle for structure 36: 156.3355289224599 degrees.
Measured bend angle for structure 37: 153.31164398786262 degrees.
Measured bend angle for structure 38: 150.78240128880736 degrees.
Measured bend angle for structure 39: 147.77243527022569 degrees.
Measured bend angle for structure 40: 149.57322871987625 degrees.
Measured bend angle for structure 41: 143.39460170413378 degrees.
Measured bend angle for structure 42: 149.01093592869722 degrees.
Measured bend angle for structure 43: 157.933707855619 degrees.
Measured bend angle for structure 44: 152.23933509307125 degrees.
Step 6: Evaluating fitness of mutants...
Evaluated fitness of mutants.
Fitness scores: [2.3833517923338974, 27.603560529526945, 19.505004752407473, 9.669591773940454, 20.806867836647527, 15.844746782835983, 3.6895670714868203, 23.71254326219176, 12.057060089993286, 0.9005496549661132, 8.175357323649024, 40.99188507498124, 40.36547779671014, 